# Stimulus selection for two-word reversible phrases

Here we will use the [Google Books N-gram dataset](https://books.google.com/ngrams/info) to select two-word phrases which are appropriate for our stimuli in the experiment. The lists of 2-grams and 1-grams used are generated by [code made publicly available by GitHub user `orgtre`](https://github.com/orgtre/google-books-ngram-frequency). The phrases need to be:

 - Comprised of two $N$ to $K$ letter words
 - The first word must be a determiner or adjective
 - The second word must be a noun
 - Commonly used, but does not appear in the top $M$ most frequent word bigrams

A list of the 45,000 most common words in the dataset and their accompanying part-of-sentence identifier will be used to subselect common words, and a list of the 100,000 bigrams will be used as candidates for the final phrases. We will sub-select from these bigrams according to the above criteria.

We begin by defining our min/max characters. We then load in the CSV files and separate the words and part-of-sentence tags for the 1-grams. We also separate the two phrase words of the bigrams.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display

CHARACTER_BOUNDS = (3, 5)
RANKLIMIT = 20_000

# Load the 45k most common words and 100k most common 2-grams
datapath = Path("../word_ngrams/").resolve()
n2grams = pd.read_csv(datapath / "2grams_english_1a_no_pos.csv").convert_dtypes()
wordcats = pd.read_csv(datapath / "1grams_english_1b_with_pos.csv").convert_dtypes()

# Isolate the words which are nouns in the top 45k
wordcats["word"] = wordcats["ngram"].str.split("_").str[0]
wordcats["POS"] = wordcats["ngram"].str.split("_").str[1]
wordcats.drop(columns=["ngram"], inplace=True)
wordcats = wordcats.reindex(columns=["word", "POS", "freq"])
nouns = wordcats[wordcats["POS"] == "NOUN"].reset_index(drop=True)
determiners = wordcats[wordcats["POS"] == "DET"].reset_index(drop=True)
adjectives = wordcats[wordcats["POS"] == "ADJ"].reset_index(drop=True)

# Split the 2-grams into the component words
n2grams["word1"] = n2grams["ngram"].str.split(" ").str[0]
n2grams["word2"] = n2grams["ngram"].str.split(" ").str[1]
n2grams.drop(columns=["ngram"], inplace=True)
n2grams["pct"] = n2grams["freq"] / n2grams["freq"].sum()
n2grams = n2grams.reindex(columns=["word1", "word2", "freq", "pct"]).convert_dtypes()

In [2]:
display(nouns)
display(n2grams)

,word,POS,freq
0,time,NOUN,344137821
1,way,NOUN,200669012
2,people,NOUN,199633836
3,life,NOUN,160230516
4,man,NOUN,149700258
...,...,...,...
27443,Gallimard,NOUN,207112
27444,lichen,NOUN,207107
27445,hack,NOUN,207101
27446,CULTURE,NOUN,207071


,word1,word2,freq,pct
0,of,the,1741529442,0.0156
1,in,the,1043688187,0.009349
2,to,the,686570630,0.00615
3,on,the,446536709,0.004
4,and,the,446190715,0.003997
...,...,...,...,...
99995,cell,adhesion,187605,0.000002
99996,and,defence,187605,0.000002
99997,the,testator,187603,0.000002
99998,suppressed,by,187590,0.000002


Next we will isolate the bigrams which are composed of any first word followed by a noun in the top 45,000 between $N$ and $K$ letters long:

In [3]:
# Isolate the 2-grams which are composed of any word and then a top-45k noun, and N-K characters
noun_2grams = n2grams[
    n2grams["word2"].isin(nouns["word"]) & n2grams["word2"].str.len().between(*CHARACTER_BOUNDS)
].copy()
display(noun_2grams)

,word1,word2,freq,pct
0,of,the,1741529442,0.0156
1,in,the,1043688187,0.009349
2,to,the,686570630,0.00615
3,on,the,446536709,0.004
4,and,the,446190715,0.003997
...,...,...,...,...
99973,Introduction,This,187641,0.000002
99975,her,round,187635,0.000002
99977,the,RNA,187634,0.000002
99979,computer,games,187629,0.000002


Create word-indexed lists of rank within a category for nouns, adjectives, and determiners. I.e. key-value mapping in which the is the word, and can be used to retrieve the rank as the value

In [4]:
idxnouns = nouns.set_index("word")
idxadj = adjectives.set_index("word")
idxdet = determiners.set_index("word")
idxnouns["rank"] = np.arange(len(idxnouns))
idxadj["rank"] = np.arange(len(idxadj))
idxdet["rank"] = np.arange(len(idxdet))
noun_2grams["w2_noun_rank"] = noun_2grams["word2"].map(idxnouns["rank"])

Next we select the phrases which start with an adjective or determiner which is between $N$ and $K$ characters long, and sort the final phrases by frequency, and save to the `phrases_candidates.csv` file:

In [5]:
# Isolate the word2 noun 2-grams that start with a determiner or adjective 4-6 char long
singlecat = wordcats.drop_duplicates(subset=["word"]).copy()
noun_2grams.apply
noun_2grams["word1_POS"] = noun_2grams["word1"].map(singlecat.set_index("word")["POS"])
detadj_2grams = (
    noun_2grams[
        noun_2grams["word1_POS"].isin(["DET", "ADJ"])
        & noun_2grams["word1"].isin(singlecat["word"])
        & noun_2grams["word1"].str.len().between(*CHARACTER_BOUNDS)
    ]
    .copy()
    .sort_values("freq", ascending=False)
)


# Add in the ranks for the first word within its category
def safe_adjdet_rank(row):
    try:
        return idxadj.loc[row["word1"]]["rank"]
    except KeyError:
        return idxdet.loc[row["word1"]]["rank"]


detadj_2grams["w1_adjdet_rank"] = detadj_2grams.apply(safe_adjdet_rank, axis=1)

detadj_2grams.to_csv(datapath / "phrases_candidates.csv", index=False)
detadj_2grams = detadj_2grams.reset_index().rename(columns={"index": "ngram_rank"})
display(detadj_2grams)


,ngram_rank,word1,word2,freq,pct,w2_noun_rank,word1_POS,w1_adjdet_rank
0,58,all,the,81315406,0.000728,20329,DET,595
1,95,the,world,61563770,0.000551,11,DET,535
2,109,the,time,56081273,0.000502,0,DET,535
3,129,the,way,49869616,0.000447,1,DET,535
4,168,the,two,43138107,0.000386,5270,DET,535
...,...,...,...,...,...,...,...,...
4112,99915,open,water,187739,0.000002,27,ADJ,48
4113,99946,right,path,187686,0.000002,526,ADJ,25
4114,99954,both,male,187670,0.000002,1343,DET,17
4115,99977,the,RNA,187634,0.000002,3854,DET,535


Finally we want to know that the phrases are also _not reversible_ so that they can successfully be used for the non-phrase condition in `word2, word1` order.

This isn't trivial, so as a rough proxy (we will hand-select the final stimuli anyways) we look to ensure the candidate phrases are not present in reverse order in the top 100k bigrams.

We also exclude elements from the M most common bigrams, so that subjects don't end up processing the phrase as a single non-syntactic package

In [6]:
# Create a copy of the data in all-lowercase form, and another reversed copy of that lowercase form
lower = detadj_2grams.copy()
lower["word1"], lower["word2"] = lower["word1"].str.lower(), lower["word2"].str.lower()
rev_lower = lower.copy().rename(columns={"word1": "word2", "word2": "word1"})

# Set the index of both dataframes to the 2-gram words for comparison
rev_lower.set_index(["word1", "word2"], inplace=True)
lower.set_index(["word1", "word2"], inplace=True)
nonreversible = lower.index.difference(rev_lower.index)
final_candidates = lower.loc[nonreversible].sort_values("word2").reset_index()
final_candidates = final_candidates.query("ngram_rank >= @RANKLIMIT")
# final_candidates.set_index(["word2", "word1"], drop=False, inplace=True)
# final_candidates.index.names = ["noun", "adj-det"]
final_candidates = final_candidates.reindex(
    columns=[
        "ngram_rank",
        "pct",
        "word1",
        "word2",
        "w1_POS",
        "w1_adjdet_rank",
        "w2_noun_rank",
        "rev_word1",
        "rev_word2",
    ]
)

pd.set_option("display.max_rows", 20)
display(f"The top 20 most used candidates out of {len(lower)}:")
display(final_candidates.head(20))
final_candidates.to_csv(datapath / "phrases_final_candidates.csv", index=False)


'The top 20 most used candidates out of 4117:'

,ngram_rank,pct,word1,word2,w1_POS,w1_adjdet_rank,w2_noun_rank,rev_word1,rev_word2
0,79155,0.000002,the,abbey,NaN,535,8037,NaN,NaN
1,62163,0.000003,the,abbey,NaN,535,15094,NaN,NaN
2,89891,0.000002,the,abbot,NaN,535,16648,NaN,NaN
4,45523,0.000004,this,about,NaN,4,25099,NaN,NaN
5,52153,0.000003,bad,about,NaN,84,25099,NaN,NaN
6,41107,0.000004,clear,about,NaN,57,25099,NaN,NaN
7,79845,0.000002,crazy,about,NaN,480,25099,NaN,NaN
8,36004,0.000005,good,about,NaN,7,25099,NaN,NaN
9,32300,0.000005,happy,about,NaN,122,25099,NaN,NaN
10,50672,0.000003,less,about,NaN,63,25099,NaN,NaN


# Consonant clusters

We want to create stimuli of similar length to the phrase words which are not words and are [unlikely to be "illusory" vowel interpretations](https://www.sciencedirect.com/science/article/pii/S0095447011001057). We will aim for 4 character strings comprised of 2 infrequent letter bigrams, and manually filter these.

We will use a computed bigram frequency of _letters_ rather than words to determine the consonant clusters used for the non-word condition of both blocks of experiments. [These frequencies were computed by Peter Norvig and published in SAS' blog by Rick Wicklin](https://blogs.sas.com/content/iml/2014/09/26/bigrams.html). We use the blog post as the original article no longer has the data available.

We directly import the string from Wicklin's SAS program and mangle it a bit to get it into a DataFrame.

In [7]:
rawdatastr = """
AA 0.003% BA 0.146% CA 0.538% DA 0.151% EA 0.688% FA 0.164% GA 0.148% HA 0.926% IA 0.286% JA 0.026% KA 0.017% LA 0.528% MA 0.565% 
NA 0.347% OA 0.057% PA 0.324% QA 0.000% RA 0.686% SA 0.218% TA 0.530% UA 0.136% VA 0.140% WA 0.385% XA 0.030% YA 0.016% ZA 0.025% 
AB 0.230% BB 0.011% CB 0.001% DB 0.003% EB 0.027% FB 0.000% GB 0.000% HB 0.004% IB 0.099% JB 0.000% KB 0.001% LB 0.007% MB 0.090%
NB 0.004% OB 0.097% PB 0.001% QB 0.000% RB 0.027% SB 0.008% TB 0.003% UB 0.089% VB 0.000% WB 0.001% XB 0.000% YB 0.004% ZB 0.000%
AC 0.448% BC 0.002% CC 0.083% DC 0.003% EC 0.477% FC 0.001% GC 0.000% HC 0.001% IC 0.699% JC 0.000% KC 0.000% LC 0.012% MC 0.004%
NC 0.416% OC 0.166% PC 0.001% QC 0.000% RC 0.121% SC 0.155% TC 0.026% UC 0.188% VC 0.000% WC 0.001% XC 0.026% YC 0.014% ZC 0.000%
AD 0.368% BD 0.002% CD 0.002% DD 0.043% ED 1.168% FD 0.000% GD 0.003% HD 0.003% ID 0.296% JD 0.000% KD 0.001% LD 0.253% MD 0.001% 
ND 1.352% OD 0.195% PD 0.001% QD 0.000% RD 0.189% SD 0.005% TD 0.001% UD 0.091% VD 0.000% WD 0.004% XD 0.000% YD 0.007% ZD 0.000%
AE 0.012% BE 0.576% CE 0.651% DE 0.765% EE 0.378% FE 0.237% GE 0.385% HE 3.075% IE 0.385% JE 0.052% KE 0.214% LE 0.829% ME 0.793%
NE 0.692% OE 0.039% PE 0.478% QE 0.000% RE 1.854% SE 0.932% TE 1.205% UE 0.147% VE 0.825% WE 0.361% XE 0.022% YE 0.093% ZE 0.050%
AF 0.074% BF 0.000% CF 0.001% DF 0.003% EF 0.163% FF 0.146% GF 0.001% HF 0.002% IF 0.203% JF 0.000% KF 0.002% LF 0.053% MF 0.004%
NF 0.067% OF 1.175% PF 0.001% QF 0.000% RF 0.032% SF 0.017% TF 0.006% UF 0.019% VF 0.000% WF 0.002% XF 0.002% YF 0.001% ZF 0.000%
AG 0.205% BG 0.000% CG 0.001% DG 0.031% EG 0.120% FG 0.001% GG 0.025% HG 0.000% IG 0.255% JG 0.000% KG 0.003% LG 0.006% MG 0.001%
NG 0.953% OG 0.094% PG 0.000% QG 0.000% RG 0.100% SG 0.002% TG 0.002% UG 0.128% VG 0.000% WG 0.000% XG 0.000% YG 0.003% ZG 0.000%
AH 0.014% BH 0.001% CH 0.598% DH 0.005% EH 0.026% FH 0.000% GH 0.228% HH 0.001% IH 0.002% JH 0.000% KH 0.003% LH 0.002% MH 0.001% 
NH 0.011% OH 0.021% PH 0.094% QH 0.000% RH 0.015% SH 0.315% TH 3.556% UH 0.001% VH 0.000% WH 0.379% XH 0.004% YH 0.001% ZH 0.001%
AI 0.316% BI 0.107% CI 0.281% DI 0.493% EI 0.183% FI 0.285% GI 0.152% HI 0.763% II 0.023% JI 0.003% KI 0.098% LI 0.624% MI 0.318%
NI 0.339% OI 0.088% PI 0.123% QI 0.000% RI 0.728% SI 0.550% TI 1.343% UI 0.101% VI 0.270% WI 0.374% XI 0.039% YI 0.029% ZI 0.012%
AJ 0.012% BJ 0.023% CJ 0.000% DJ 0.005% EJ 0.005% FJ 0.000% GJ 0.000% HJ 0.000% IJ 0.001% JJ 0.000% KJ 0.000% LJ 0.000% MJ 0.000%
NJ 0.011% OJ 0.007% PJ 0.000% QJ 0.000% RJ 0.001% SJ 0.000% TJ 0.000% UJ 0.001% VJ 0.000% WJ 0.000% XJ 0.000% YJ 0.000% ZJ 0.000%
AK 0.105% BK 0.000% CK 0.118% DK 0.000% EK 0.016% FK 0.000% GK 0.000% HK 0.000% IK 0.043% JK 0.000% KK 0.000% LK 0.020% MK 0.000% 
NK 0.052% OK 0.064% PK 0.001% QK 0.000% RK 0.097% SK 0.039% TK 0.000% UK 0.005% VK 0.000% WK 0.001% XK 0.000% YK 0.000% ZK 0.000%
AL 1.087% BL 0.233% CL 0.149% DL 0.032% EL 0.530% FL 0.065% GL 0.061% HL 0.013% IL 0.432% JL 0.000% KL 0.011% LL 0.577% ML 0.005%
NL 0.064% OL 0.365% PL 0.263% QL 0.000% RL 0.086% SL 0.056% TL 0.098% UL 0.346% VL 0.000% WL 0.015% XL 0.001% YL 0.015% ZL 0.001%
AM 0.285% BM 0.003% CM 0.003% DM 0.018% EM 0.374% FM 0.001% GM 0.010% HM 0.013% IM 0.318% JM 0.000% KM 0.002% LM 0.023% MM 0.096%
NM 0.028% OM 0.546% PM 0.016% QM 0.000% RM 0.175% SM 0.065% TM 0.026% UM 0.138% VM 0.000% WM 0.001% XM 0.000% YM 0.024% ZM 0.000%
AN 1.985% BN 0.002% CN 0.001% DN 0.008% EN 1.454% FN 0.000% GN 0.066% HN 0.026% IN 2.433% JN 0.000% KN 0.051% LN 0.006% MN 0.009%
NN 0.073% ON 1.758% PN 0.001% QN 0.000% RN 0.160% SN 0.009% TN 0.010% UN 0.394% VN 0.000% WN 0.079% XN 0.000% YN 0.013% ZN 0.000%
AO 0.005% BO 0.195% CO 0.794% DO 0.188% EO 0.073% FO 0.488% GO 0.132% HO 0.485% IO 0.835% JO 0.054% KO 0.006% LO 0.387% MO 0.337%
NO 0.465% OO 0.210% PO 0.361% QO 0.000% RO 0.727% SO 0.398% TO 1.041% UO 0.011% VO 0.071% WO 0.222% XO 0.003% YO 0.150% ZO 0.007%
AP 0.203% BP 0.001% CP 0.001% DP 0.002% EP 0.172% FP 0.000% GP 0.000% HP 0.001% IP 0.089% JP 0.000% KP 0.001% LP 0.019% MP 0.239%
NP 0.006% OP 0.224% PP 0.137% QP 0.000% RP 0.042% SP 0.191% TP 0.004% UP 0.136% VP 0.000% WP 0.001% XP 0.067% YP 0.025% ZP 0.000%
AQ 0.002% BQ 0.000% CQ 0.005% DQ 0.001% EQ 0.057% FQ 0.000% GQ 0.000% HQ 0.000% IQ 0.011% JQ 0.000% KQ 0.000% LQ 0.000% MQ 0.000%
NQ 0.006% OQ 0.001% PQ 0.000% QQ 0.000% RQ 0.001% SQ 0.007% TQ 0.000% UQ 0.000% VQ 0.000% WQ 0.000% XQ 0.000% YQ 0.000% ZQ 0.000%
AR 1.075% BR 0.112% CR 0.149% DR 0.085% ER 2.048% FR 0.213% GR 0.197% HR 0.084% IR 0.315% JR 0.000% KR 0.003% LR 0.010% MR 0.003%
NR 0.009% OR 1.277% PR 0.474% QR 0.000% RR 0.121% SR 0.006% TR 0.426% UR 0.543% VR 0.001% WR 0.031% XR 0.000% YR 0.008% ZR 0.000%
AS 0.871% BS 0.046% CS 0.023% DS 0.126% ES 1.339% FS 0.006% GS 0.051% HS 0.015% IS 1.128% JS 0.000% KS 0.048% LS 0.142% MS 0.093%
NS 0.509% OS 0.290% PS 0.055% QS 0.000% RS 0.397% SS 0.405% TS 0.337% US 0.454% VS 0.001% WS 0.035% XS 0.000% YS 0.097% ZS 0.000%
AT 1.487% BT 0.017% CT 0.461% DT 0.003% ET 0.413% FT 0.082% GT 0.015% HT 0.130% IT 1.123% JT 0.000% KT 0.001% LT 0.124% MT 0.001%
NT 1.041% OT 0.442% PT 0.106% QT 0.000% RT 0.362% ST 1.053% TT 0.171% UT 0.405% VT 0.000% WT 0.007% XT 0.047% YT 0.017% ZT 0.000%
AU 0.119% BU 0.185% CU 0.163% DU 0.148% EU 0.031% FU 0.096% GU 0.086% HU 0.074% IU 0.017% JU 0.059% KU 0.003% LU 0.135% MU 0.115%
NU 0.079% OU 0.870% PU 0.105% QU 0.148% RU 0.128% SU 0.311% TU 0.255% UU 0.001% VU 0.002% WU 0.001% XU 0.005% YU 0.001% ZU 0.002%
AV 0.205% BV 0.004% CV 0.000% DV 0.019% EV 0.255% FV 0.000% GV 0.000% HV 0.000% IV 0.288% JV 0.000% KV 0.000% LV 0.035% MV 0.000%
NV 0.052% OV 0.178% PV 0.000% QV 0.000% RV 0.069% SV 0.001% TV 0.001% UV 0.003% VV 0.000% WV 0.000% XV 0.002% YV 0.000% ZV 0.000%
AW 0.060% BW 0.000% CW 0.000% DW 0.008% EW 0.117% FW 0.000% GW 0.001% HW 0.005% IW 0.001% JW 0.000% KW 0.002% LW 0.013% MW 0.001%
NW 0.006% OW 0.330% PW 0.001% QW 0.000% RW 0.013% SW 0.024% TW 0.082% UW 0.000% VW 0.000% WW 0.000% XW 0.000% YW 0.003% ZW 0.000%
AX 0.019% BX 0.000% CX 0.000% DX 0.000% EX 0.214% FX 0.000% GX 0.000% HX 0.000% IX 0.022% JX 0.000% KX 0.000% LX 0.000% MX 0.000%
NX 0.003% OX 0.019% PX 0.000% QX 0.000% RX 0.001% SX 0.000% TX 0.000% UX 0.004% VX 0.000% WX 0.000% XX 0.003% YX 0.000% ZX 0.000%
AY 0.217% BY 0.176% CY 0.042% DY 0.050% EY 0.144% FY 0.009% GY 0.026% HY 0.050% IY 0.000% JY 0.000% KY 0.006% LY 0.425% MY 0.062%
NY 0.098% OY 0.036% PY 0.012% QY 0.000% RY 0.248% SY 0.057% TY 0.227% UY 0.005% VY 0.005% WY 0.002% XY 0.003% YY 0.000% ZY 0.002%
AZ 0.012% BZ 0.000% CZ 0.001% DZ 0.000% EZ 0.005% FZ 0.000% GZ 0.000% HZ 0.000% IZ 0.064% JZ 0.000% KZ 0.000% LZ 0.000% MZ 0.000%
NZ 0.004% OZ 0.003% PZ 0.000% QZ 0.000% RZ 0.001% SZ 0.000% TZ 0.004% UZ 0.002% VZ 0.000% WZ 0.000% XZ 0.000% YZ 0.002% ZZ 0.003%"""

In [9]:
rawdatastr = rawdatastr.replace("%", "").replace("\n", " ").replace("  ", " ")  # Clean up the data
data_arr = np.array([item.lower() for item in rawdatastr.split(" ") if len(item) > 0]).reshape(
    -1, 2
)  # Split the data into an array
letter2gram = pd.DataFrame(
    data_arr, columns=["letter_bigram", "pct_freq"]
)  # Convert to a DataFrame

Let's first remove all bigrams which contain vowels (or "y"), and then look at the 25 least frequent consonant letter bigrams to see what we can compose from these infrequent strings. Note that the frequency is measures as a percentage of occurrance among letter bigrams, and is in steps of 0.001%

In [13]:
def has_vowel(s):
    return any(c in "aeiouy" for c in s)


mask = letter2gram["letter_bigram"].apply(has_vowel)
cons_2grams = letter2gram[~mask].copy()
cons_2grams = cons_2grams.sort_values("pct_freq").reset_index(drop=True)
pd.set_option("display.max_rows", 50)
display(cons_2grams.head(25))

,letter_bigram,pct_freq
0,zm,0.000
1,hg,0.000
2,jg,0.000
3,tq,0.000
4,qq,0.000
5,pq,0.000
6,pg,0.000
7,qg,0.000
8,mq,0.000
9,vg,0.000


All of these look like valid candidates with the exception of "qq", perhaps, as we would like to avoid stimuli that could be seen as a "list of characters" rather than an invalid word-like structure. 

After removing "qq", lets look at the possible pairings of the the 24 least-common entries ($24!$ total pairings).

In [28]:
from itertools import product

pairclusters = (
    cons_2grams.drop(index=4).head(24)["letter_bigram"].values
)  # only the 10 least freq minus qq
dibigrams = pd.DataFrame(list(product(pairclusters, repeat=2)), columns=["clus1", "clus2"])
dibigrams = dibigrams[dibigrams["clus1"] != dibigrams["clus2"]].copy()
dibigrams["non_word"] = dibigrams["clus1"] + dibigrams["clus2"]
dibigrams.set_index(["clus1", "clus2"], inplace=True)
dibigrams.sort_index(inplace=True)

pd.set_option("display.max_rows", 100)
display(f"{len(dibigrams)} total pairings")
display(dibigrams.head(100))

'552 total pairings'

non_word
clus1 clus2         
bg    fh        bgfh
      fq        bgfq
      hg        bghg
      hq        bghq
      jg        bgjg
      jq        bgjq
      kq        bgkq
      kv        bgkv
      lq        bglq
      mq        bgmq
      pg        bgpg
      pq        bgpq
      qg        bgqg
      tq        bgtq
      vg        bgvg
      vq        bgvq
      wg        bgwg
      wq        bgwq
      xg        bgxg
      zf        bgzf
      zg        bgzg
      zm        bgzm
      zr        bgzr
fh    bg        fhbg
      fq        fhfq
      hg        fhhg
      hq        fhhq
      jg        fhjg
      jq        fhjq
      kq        fhkq
      kv        fhkv
      lq        fhlq
      mq        fhmq
      pg        fhpg
      pq        fhpq
      qg        fhqg
      tq        fhtq
      vg        fhvg
      vq        fhvq
      wg        fhwg
      wq        fhwq
      xg        fhxg
      zf        fhzf
      zg        fhzg
      zm        fhzm
      zr        fhzr
fq    bg        fqbg
      fh        fqfh
      hg        fqhg
      hq        fqhq
      jg        fqjg
      jq        fqjq
      kq        fqkq
      kv        fqkv
      lq        fqlq
      mq        fqmq
      pg        fqpg
      pq        fqpq
      qg        fqqg
      tq        fqtq
      vg        fqvg
      vq        fqvq
      wg        fqwg
      wq        fqwq
      xg        fqxg
      zf        fqzf
      zg        fqzg
      zm        fqzm
      zr        fqzr
hg    bg        hgbg
      fh        hgfh
      fq        hgfq
      hq        hghq
      jg        hgjg
      jq        hgjq
      kq        hgkq
      kv        hgkv
      lq        hglq
      mq        hgmq
      pg        hgpg
      pq        hgpq
      qg        hgqg
      tq        hgtq
      vg        hgvg
      vq        hgvq
      wg        hgwg
      wq        hgwq
      xg        hgxg
      zf        hgzf
      zg        hgzg
      zm        hgzm
      zr        hgzr
hq    bg        hqbg
      fh        hqfh
      fq        hqfq
      hg        hqhg
      jg        hqjg
      jq        hqjq
      kq        hqkq
      kv        hqkv

It becomes apparent that many of the least frequent bigrams have the same second letter. Let's ensure that in the 4-character string the 2nd and 4th letters do not match, nor do the 1st and 3rd. We will also throw out those which have the inner pair (2/3) matching for fun.

We will generate the final list by taking 3 pairs from within each set of starting clusters.

In [55]:
N_PAIRS = 4


def matching_letters(s):
    return (s[0] == s[2]) | (s[1] == s[3]) | (s[1] == s[2])


mask = dibigrams["non_word"].apply(matching_letters)
final_dibigrams = dibigrams[~mask].copy()

rng = np.random.default_rng(0)
select_idx = np.zeros((N_PAIRS * len(pairclusters), 2), dtype=np.dtypes.StrDType)
select_idx[:, 0] = np.repeat(pairclusters, N_PAIRS)
for i, clus in enumerate(pairclusters):
    start = i * N_PAIRS
    stop = start + N_PAIRS
    clus_options = final_dibigrams.loc[clus, :].reset_index()["clus2"].values
    select_idx[start:stop, 1] = rng.choice(clus_options, N_PAIRS, replace=False)

rand_dbg = final_dibigrams.loc[[tuple(row) for row in select_idx]]
rand_dbg.to_csv(datapath / "cons_clust_final_candidates.csv")
print(f"{len(select_idx)} total randomly-selected pairs")
display(rand_dbg)


96 total randomly-selected pairs


non_word
clus1 clus2         
zm    jg        zmjg
      lq        zmlq
      tq        zmtq
      pg        zmpg
hg    fh        hgfh
      wq        hgwq
      jq        hgjq
      vq        hgvq
jg    tq        jgtq
      wq        jgwq
      pq        jgpq
      zm        jgzm
tq    bg        tqbg
      xg        tqxg
      wg        tqwg
      pg        tqpg
pq    xg        pqxg
      hg        pqhg
      zg        pqzg
      wg        pqwg
pg    jq        pgjq
      kv        pgkv
      lq        pglq
      fh        pgfh
qg    vq        qgvq
      fh        qgfh
      fq        qgfq
      zm        qgzm
mq    vg        mqvg
      xg        mqxg
      kv        mqkv
      pg        mqpg
vg    kq        vgkq
      pq        vgpq
      zm        vgzm
      tq        vgtq
wg    kq        wgkq
      vq        wgvq
      pq        wgpq
      fq        wgfq
xg    zf        xgzf
      kq        xgkq
      mq        xgmq
      kv        xgkv
lq    pg        lqpg
      xg        lqxg
      zf        lqzf
      kv        lqkv
zg    vq        zgvq
      lq        zglq
      kv        zgkv
      kq        zgkq
kq    hg        kqhg
      zr        kqzr
      zf        kqzf
      zm        kqzm
jq    pg        jqpg
      zg        jqzg
      kv        jqkv
      zf        jqzf
hq    zm        hqzm
      xg        hqxg
      zf        hqzf
      bg        hqbg
fh    xg        fhxg
      pq        fhpq
      wq        fhwq
      zg        fhzg
vq    pg        vqpg
      zr        vqzr
      kv        vqkv
      bg        vqbg
zr    mq        zrmq
      wg        zrwg
      wq        zrwq
      jg        zrjg
wq    zm        wqzm
      fh        wqfh
      bg        wqbg
      xg        wqxg
fq    zf        fqzf
      pg        fqpg
      zr        fqzr
      wg        fqwg
bg    wq        bgwq
      fh        bgfh
      lq        bglq
      hq        bghq
zf    xg        zfxg
      pq        zfpq
      wg        zfwg
      kq        zfkq
kv    mq        kvmq
      lq        kvlq
      tq        kvtq
      pg        kvpg